In [14]:
import os
import h5py

data_dir='/data/scratch/jiayin'
aggr_time='hour'
time_range='all'
tele_column='sms2'
file_name = "milan_{}_T_N_5.h5".format(
            'min' if aggr_time is None else aggr_time)
filePath = os.path.join(data_dir, file_name)
print(filePath)

if not os.path.exists(filePath):
    raise FileNotFoundError("file {} not found".format(filePath))


/data/scratch/jiayin/milan_hour_T_N_5.h5


In [16]:
# 虽然
import pandas as pd
with h5py.File(filePath, 'r') as f:
    print("Keys in HDF5 file:", list(f.keys()))
    # 读取数据集
    data = f['data'][:]
    time_data = f['time'][:]
    # 如果时间数据是字节类型，需要解码
    time_data = [t.decode('utf-8') if isinstance(t, bytes) else t for t in time_data]

print('data shape: ', data.shape)
# 将时间数据转换为 pandas 的 DatetimeIndex
times = pd.to_datetime(time_data, format='%Y-%m-%d %H:%M:%S')
print("Total unique timestamps:", times.nunique())

# 如果你想检查是否是一天一天排列，可以计算相邻日期之间的日期部分的差值
# 先提取日期部分
dates = times.normalize()  # 将时间归一化到午夜
diff = dates.to_series().diff().dropna()

# 查看所有不同的日期差值
print("Unique date differences:", diff.unique())

# 统计每种差值出现的次数
diff_counts = diff.value_counts().sort_index()
print("\nDate differences counts:")
for delta, count in diff_counts.items():
    print(f"{delta}: {count} times")

Keys in HDF5 file: ['data', 'time']
data shape:  (1416, 10000, 5)
Total unique timestamps: 1416
Unique date differences: [86400000000000              0]

Date differences counts:
0 days 00:00:00: 1353 times
1 days 00:00:00: 62 times


In [ ]:
# 检查时间间隔是否连续
import pandas as pd
with h5py.File(filePath, 'r') as f:
    # 假设 time 数据集存储的是字符串或字节型时间戳
    time_data = f['time'][:]  # 读取全部数据

# 如果时间数据以字节形式存储，需要先解码（假设是UTF-8编码）
time_data = [t.decode('utf-8') if isinstance(t, bytes) else t for t in time_data]

# 将时间数据转换为 pandas datetime 对象
times = pd.to_datetime(time_data, format='%Y-%m-%d %H:%M:%S')

# 将 DatetimeIndex 转换为 Series 并计算相邻时间差（Timedelta）
time_series = times.to_series()
time_diff = time_series.diff().dropna()

# 使用 value_counts 统计每个时间差出现的次数
diff_counts = time_diff.value_counts().sort_index()
print("\n各连续时间间隔及其出现次数：")
for td, count in diff_counts.items():
    # td 是 Timedelta 对象，直接转换为字符串即可获得易读格式
    print(f"{str(td)} : {count} 次")

# 定义要查找的时间间隔
gap = pd.Timedelta("1 days 00:10:00")

# 筛选出那些间隔等于 gap 的位置
gap_positions = time_diff[time_diff == gap]

print("出现 1 days 00:10:00 的时间间隔的位置和对应的时间戳：")
for idx, gap_value in gap_positions.items():
    # idx 对应当前时间戳，前一个时间戳为 time_series.loc[idx] 的前一行
    # 使用索引的 .shift() 或直接利用 index 获取前一个值：
    prev_idx = time_series.index.get_loc(idx) - 1
    if prev_idx >= 0:
        prev_time = time_series.iloc[prev_idx]
    else:
        prev_time = None
    print(f"Gap from {prev_time} to {idx} (间隔: {gap_value})")

In [ ]:
# 检查初始csv，时间间隔最小是10min
# 文件名对应的时间是混乱的，但是最后的h5文件的时间是正确排列的，还好

data_dir='/data/scratch/jiayin'
file_name = 'sms-call-internet-mi-2013-11-18.csv'
path = os.path.join(data_dir, file_name)
data = pd.read_csv(path, header=0, index_col=0)
data.reset_index(inplace=True)  # 重置索引，将 'cellid' 恢复为普通列
data['time'] = pd.to_datetime(data['time'], unit='ms')  # 原时间格式为ms
print(data[10:15])

In [ ]:
import matplotlib.pyplot as plt
# 国家代码列表示这个电话/短信发生的国家。从统计结果看，最多的国家代码是 39（意大利），其次是 0（可能表示国内或未记录国家信息），再是 33（法国）、49（德国）和 46（瑞典）。似乎没有提供什么信息，故舍弃了。

# 统计各 countrycode 的频次
counts = data['countrycode'].value_counts()
# 输出频次最高的 countrycode 和出现次数
print("出现最多的 countrycode：")
print(counts.head(5))

# 设置图形大小
plt.figure(figsize=(12, 6))
# 绘制柱状图
counts.plot(kind='bar')

plt.xlabel("Country Code")
plt.ylabel("Frequency")
plt.title("Frequency of Each Country Code")
plt.tight_layout()
plt.show()